In [2]:
import pandas as pd
import numpy as np
import pickle
import scanpy as sc
import sys
sys.path.append('../GENIE3/GENIE3_python/')
from GENIE3 import GENIE3
from matplotlib import pyplot as plt

In [3]:
data = pd.read_table('../Data/SARS-CoV-2/gene_expression_matrix.tsv')
data

,gene_id,Sample_10_S19,Sample_11_S20,Sample_12_S21,Sample_13_S22,Sample_14_S23,Sample_15_S24,Sample_16_S25,Sample_17_S26,Sample_18_S27,...,Sample_67_S41,Sample_68_S42,Sample_69_S43,Sample_6_S15,Sample_70_S44,Sample_71_S45,Sample_72_S46,Sample_7_S16,Sample_8_S17,Sample_9_S18
0,ENSG00000000003,77.72,93.20,48.05,75.24,92.28,82.73,77.35,86.75,56.45,...,80.47,89.92,67.71,62.96,63.60,68.11,69.46,110.70,102.62,98.76
1,ENSG00000000005,0.00,0.00,0.00,0.00,0.04,0.00,0.04,0.04,0.05,...,0.00,0.04,0.04,0.06,0.09,0.00,0.04,0.00,0.00,0.00
2,ENSG00000000419,47.65,55.06,44.20,49.93,53.49,50.49,51.70,48.89,46.14,...,46.32,54.36,49.08,51.32,48.48,41.96,46.77,53.76,56.65,52.65
3,ENSG00000000457,15.02,15.99,12.40,21.41,21.17,19.50,20.26,18.52,14.56,...,15.44,16.98,18.03,13.40,16.35,16.22,16.79,17.35,16.64,16.88
4,ENSG00000000460,2.30,2.88,3.05,1.67,2.85,2.28,2.35,2.34,2.79,...,2.14,1.90,1.87,2.57,1.73,2.85,2.22,2.17,2.55,2.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60666,ENSG00000288640,0.66,0.06,1.51,0.11,0.31,0.50,0.52,0.51,0.88,...,0.37,0.43,0.98,0.86,0.45,0.89,0.72,0.59,0.44,0.14
60667,ENSG00000288642,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
60668,ENSG00000288643,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
60669,ENSG00000288644,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [4]:
data = data.set_index('gene_id')
data

,Sample_10_S19,Sample_11_S20,Sample_12_S21,Sample_13_S22,Sample_14_S23,Sample_15_S24,Sample_16_S25,Sample_17_S26,Sample_18_S27,Sample_19_S28,...,Sample_67_S41,Sample_68_S42,Sample_69_S43,Sample_6_S15,Sample_70_S44,Sample_71_S45,Sample_72_S46,Sample_7_S16,Sample_8_S17,Sample_9_S18
gene_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,77.72,93.20,48.05,75.24,92.28,82.73,77.35,86.75,56.45,92.25,...,80.47,89.92,67.71,62.96,63.60,68.11,69.46,110.70,102.62,98.76
ENSG00000000005,0.00,0.00,0.00,0.00,0.04,0.00,0.04,0.04,0.05,0.00,...,0.00,0.04,0.04,0.06,0.09,0.00,0.04,0.00,0.00,0.00
ENSG00000000419,47.65,55.06,44.20,49.93,53.49,50.49,51.70,48.89,46.14,51.89,...,46.32,54.36,49.08,51.32,48.48,41.96,46.77,53.76,56.65,52.65
ENSG00000000457,15.02,15.99,12.40,21.41,21.17,19.50,20.26,18.52,14.56,15.26,...,15.44,16.98,18.03,13.40,16.35,16.22,16.79,17.35,16.64,16.88
ENSG00000000460,2.30,2.88,3.05,1.67,2.85,2.28,2.35,2.34,2.79,2.26,...,2.14,1.90,1.87,2.57,1.73,2.85,2.22,2.17,2.55,2.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000288640,0.66,0.06,1.51,0.11,0.31,0.50,0.52,0.51,0.88,0.31,...,0.37,0.43,0.98,0.86,0.45,0.89,0.72,0.59,0.44,0.14
ENSG00000288642,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ENSG00000288643,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [8]:
# Get raw expression count matrix
exp_matrix = data.to_numpy().T
obs = data.columns.values

# with open('../Data/raw_exp_matrix.pickle', 'wb') as f:
#     pickle.dump(exp_matrix, f)

In [9]:
exp_matrix.shape

(70, 60671)

In [10]:
# Get highly variable genes 

# Creating AnnData Object from
adata = sc.AnnData(X=exp_matrix, obs=obs)

# with open('../Data/SARS-CoV-2/gene_exp_anndata.pickle', 'wb') as f:
#     pickle.dump(adata, f)

In [11]:
# log-transforming data so we can find highly variable genes
sc.pp.log1p(adata)

In [12]:
# Find highly variable genes
sc.pp.highly_variable_genes(adata, n_top_genes=500)

In [13]:
# Getting top 500 most highly variable genes
highly_variable_idx = [i for i in range(adata.X.shape[1]) if adata.var['highly_variable'].iloc[i] == True]
highly_variable_idx

[61,
 315,
 389,
 559,
 578,
 992,
 1239,
 1519,
 1865,
 1929,
 1967,
 2124,
 2909,
 3094,
 3321,
 3398,
 3882,
 4421,
 4765,
 4979,
 5314,
 5813,
 5828,
 6016,
 6019,
 6176,
 6270,
 6417,
 6711,
 6842,
 6913,
 7014,
 7309,
 7528,
 7547,
 7621,
 8285,
 8371,
 8423,
 9232,
 10382,
 10437,
 10578,
 10592,
 10938,
 10940,
 11595,
 11902,
 12148,
 12223,
 12546,
 12549,
 12594,
 12595,
 12604,
 12861,
 13032,
 13057,
 13190,
 13829,
 14604,
 15036,
 15973,
 16317,
 16321,
 16486,
 16558,
 16653,
 16949,
 17504,
 17801,
 17885,
 17978,
 17979,
 18049,
 18050,
 18053,
 18071,
 18146,
 18147,
 18157,
 18159,
 18172,
 18173,
 18179,
 18183,
 18217,
 18230,
 18240,
 18272,
 18284,
 18297,
 18301,
 18318,
 18333,
 18379,
 18418,
 18433,
 18439,
 18488,
 18502,
 18519,
 18529,
 18542,
 18548,
 18557,
 18576,
 18578,
 18592,
 18608,
 18615,
 18659,
 18664,
 18665,
 18698,
 18718,
 18723,
 18725,
 18735,
 18749,
 18759,
 18826,
 18855,
 18864,
 18866,
 18884,
 18897,
 18907,
 18950,
 18983,
 19005,

In [41]:
highly_variable_genes = data.iloc[highly_variable_idx].index.to_list()
highly_variable_genes

['ENSG00000004809',
 'ENSG00000012223',
 'ENSG00000019186',
 'ENSG00000039537',
 'ENSG00000041982',
 'ENSG00000067048',
 'ENSG00000073756',
 'ENSG00000081041',
 'ENSG00000090104',
 'ENSG00000091513',
 'ENSG00000092295',
 'ENSG00000099860',
 'ENSG00000104368',
 'ENSG00000105388',
 'ENSG00000106538',
 'ENSG00000107201',
 'ENSG00000111331',
 'ENSG00000115414',
 'ENSG00000117983',
 'ENSG00000119922',
 'ENSG00000122861',
 'ENSG00000126368',
 'ENSG00000126549',
 'ENSG00000128383',
 'ENSG00000128422',
 'ENSG00000129824',
 'ENSG00000130487',
 'ENSG00000131203',
 'ENSG00000133106',
 'ENSG00000134184',
 'ENSG00000134531',
 'ENSG00000135114',
 'ENSG00000136689',
 'ENSG00000137801',
 'ENSG00000137841',
 'ENSG00000138166',
 'ENSG00000142871',
 'ENSG00000143369',
 'ENSG00000143556',
 'ENSG00000149591',
 'ENSG00000159763',
 'ENSG00000160201',
 'ENSG00000161133',
 'ENSG00000161281',
 'ENSG00000163209',
 'ENSG00000163216',
 'ENSG00000165474',
 'ENSG00000166670',
 'ENSG00000167656',
 'ENSG00000167916',


In [21]:
with open('../Data/SARS-CoV-2/highly_variable_genes_500.pickle', 'wb') as f:
    pickle.dump(highly_variable_genes, f)

In [40]:
# Subsetting expression data to highly variable genes
highly_variable_genes_matrix = data.iloc[highly_variable_idx].to_numpy().T

with open('../Data/SARS-CoV-2/highly_variable_expression_matrix_501.pickle', 'wb') as f:
    pickle.dump(highly_variable_genes_matrix, f)
